# Read me for Speed of Shocks in the MSH
_Christopher Butcher_

_Emails:_
- _cmmb503@york.ac.uk_
- _thechrisbutcher@gmail.com_

_Last Updated: 13/09/2023_

---

## Introduction
I have written this notebook to try and have some of the code I wrote and used during my internship not go to waste. You will likely notice that there are lots of spelling errors in my code and my writing, so sorry in advance.

Below Is a contents table to help you find any bits of code that may be useful. I will also try and explain how to get some other peoples code to work (De Welle's code).

For more Info/code please email me (as stated above) or ask Clément to contact me. I have some code I wrote late into the internship that calculates the non-local f-value for measured shocks inside the MSH (where measurments come from Val's code). If this is something you require/want please do get in touch as it should save you some time. 

# Table of Contents

1. [g-value code (B. Michotte de Welle)](#g_val_code)
    1. [MSH Boundaries](#MSHBound)
        1. [Plotting MSH Boundaries](#boundPlot)
    2. [g-value at a point](#g_val_point)
        1. [Note on Dynamic Pressure](#dynamic_pressure)
2. [My Modules](#MyModules)
    1. [Coordinate_Systems.py](#Coordinatesystems)
    2. [Shocks_propagation.py](#shocks_prop_mod)
3. [V2_Rand_prop_points.py](#rand_point)
4. [Berdichevsky Shock json file](#shocks_file)

## g-value code (B. Michotte de Welle) <a name="g_val_code"></a>
_There is a tutorial for this code that you may have seen (I should have attached it this folder sent, tutorial_utilisation_knn.ipynb). I will try and explain its applications here though._

I used this code to map out the Boundaries of the Bow Shock and the Magnetopause. I also used it to find the value of $\vec{g}(\vec{r})$ which is defined for the bulk velocity as the following.
$$\vec{g}(\vec{r}) = \frac{\vec{u}_{msh}}{|\vec{u}_{sw}|}$$

---

### MSH Boundaries <a name="MSHBound"></a>
The geometry of the Bow shock and Magnetopause often comes in handy seeing as they are the magnetosheaths boundaries.

You require the following imports/modules to find the boundaries of the MSH.

To get the space module:
1. download the "space" package from git : git clone https://github.com/LaboratoryOfPlasmaPhysics/space
2. import the package (cf the cell below, where you may need to adapt the "path")

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
# -
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import sklearn
# -
# - Space Module imports (May need to change path name)
sys.path.append('/Users/christopherbutcher/Downloads/Summer Masterclass/space')
import space
from space.models import planetary as smp
from space import smath as sm
from space.coordinates import coordinates as scc
from space import utils as su
# - 
# - Paste this code for def of msh. prefix
#
# magnetosheath model using shue98 et jelinek2012
msh = smp.Magnetosheath(magnetopause='mp_shue1998', bow_shock ='bs_jelinek2012')
# - 

ModuleNotFoundError: No module named 'sklearn'

Code examples of finding points on the Bow shock and magentopause.

In [ ]:
# The inputs are (theta,phi) - In radians!!!
# And this assumes SWI coordinate system!!

# --------------------- Bow Shock
theta = 0 ; phi = 0
bow_shock_point = msh.bow_shock(theta,phi)
print("SWI cartesian coordinate of Bow shock point: ", bow_shock_point)
# The Output is in SWI Cartesians (In units of r_E - Earth Radii)
#
# --------------------- Magnetopause
mp_point = msh.magnetopause(theta,phi)
print("SWI cartesian coordinate of magnetopause point: ", mp_point)

#### Plotting MSH Boundaries <a name="boundPlot"></a>
Here I will plot the Bow Shock (Red) and the Magnetopause (Blue) in 3D SWI coordinates.

In [ ]:
# Requires import of one of my modules (see later section on Coordinate_systems.py for more)
import Coordinate_systems as co

phi_max    = 360
phi_min    = 0
phi_step   = 20

theta_max  = 100
theta_min  = 0
theta_step = 10

x_bs=[] ; x_mp=[]
y_bs=[] ; y_mp=[]
z_bs=[] ; z_mp=[]

for theta_i in range(theta_min,theta_max,theta_step):
    for phi_i in range(phi_min,phi_max,phi_step):
        
        theta = co.deg2rad(theta_i) ## /2 to make more accurate
        phi   = co.deg2rad(phi_i)
        
        # For Bow Shock
        bs_points = msh.bow_shock(theta,phi)
        
        # For Plotting
        x_bs.append(bs_points[0])
        y_bs.append(bs_points[1])
        z_bs.append(bs_points[2])
        
        # For Magnetopause
        mp_points = msh.magnetopause(theta,phi)
        
        x_mp.append(mp_points[0])
        y_mp.append(mp_points[1])
        z_mp.append(mp_points[2])

**Tip - Making plots interactive**
Use ```%matplotlib notebook```

In [ ]:
# Plott
%matplotlib notebook

# creating figure
fig = plt.figure()
ax = fig.add_subplot(projection='3d')


ax.scatter(x_bs,y_bs,z_bs,color='red')
ax.scatter(x_mp,y_mp,z_mp,color='blue')

# setting title and labels
ax.set_title("Magnetosheath Boundaries")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
 
# displaying the plot
plt.show()

## Important note about MSH boundaries (Dynamic Pressure) <a name="dynamic_pressure"></a>

As you would expect the boundaries of the MSH (Magnetopause and Bow shock) will depend on the speed of the solar wind and the density of the solar wind (upstream of the shock). These two quantities define the dynamic pressure (DP).

The DP is defined as follows:
$$ P_D = \rho \cdot |\vec{u}_{sw}|^2$$

Often the denisty ($\rho$) is quoted in particles/cm$^3$. So to calculate the DP you must convert to SI units (kg/m$^3$) via the following equation.
$$ \rho_{SI} =  (10^{6} \cdot  1.67 \cdot 10^{-27})\cdot \rho_{cm^{-3}}$$

You must also convert the solar wind bulk velocity from km/s to m/s by multiplying by $10^3$.

Now using the SI unit values you can calculate the DP. The expected units of the DP (for De Welle's functions) will be in nano Pascals (nPa) and can be used in the functions, ```msh.bow_shock``` and ```msh.magnetopause```. This will then adjust the geometry of the two according to the DP. An example is shown below on how to do this in code.

In [ ]:
# Will explain this module later, but must be used here for calcualtion
import Coordinate_systems as co

In [ ]:
# GSE defined
u_sw_GSE = np.array([-343.3733333333334,-10.817986666666668,8.047686666666666]) * 1e3 # m/s

# Density (in particles/cm^3)
sw_density_up = 20

# Converting to (m^-3)
sw_density_up = sw_density_up * 1e6* 1.67e-27

# ---

# Dynamic pressure Calculation                      (Make Nano Pascal)
Dyn_Press = ((sw_density_up*(co.mag(u_sw_GSE))**2)) * 1e9 

print('Dynamic Pressure: ',Dyn_Press,"nPa")

### How to change MSH geometry based on Dynamic pressure

We now want to use the dynamic pressure to change the geometry of the boundaries. This can be done as shown in the code. (simply include the dynamic pressure after $\theta,\varphi$ values).

In [ ]:
print("Bow-shock nose:")

# Here is the new bow_shock nose (accounting for dynamic pressure)
bs_point_dp = msh.bow_shock(0,0,Pd=Dyn_Press)
print("accounting for dynamic pressure:",bs_point_dp)

# Original nose values not accounting for dynamic pressure (in R_E)
bs_point = msh.bow_shock(0,0)
print("NOT-accounting for dynamic pressure:",bs_point,"\n")

print("Magnetopause nose:")

# New magnetopause nose (accounting for dynamic pressure)
mp_point_dp = msh.magnetopause(0,0,Pd=Dyn_Press)
print("accounting for dynamic pressure:",mp_point_dp)

# New magnetopause nose (accounting for dynamic pressure)
mp_point = msh.magnetopause(0,0)
print("NOT-accounting for dynamic pressure:",mp_point)

## g-value at a point <a name="g_val_point"></a>
You can also use the statistical data provided by de Welle's 'KNN_swi_Vall_k50000.pkl' file to find the bulk velocity ratio (previously denoted as $\vec{g}(\vec{r})$. This 'pkl' file is massive so goodluck finding some space to store it. You will need to also have the correct ```sklearn``` package import otherwise you will get a error to do with you not having the 'EuclidianDistance' metric (something like that).

First we will begin with the imports and then the interpval function.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import sklearn

In [ ]:
# Note you will need to reference where this file is!
interpvall = pd.read_pickle('/Users/christopherbutcher/Downloads/Summer Masterclass/KNN_swi_Vall_k50000.pkl')

**If error with sklearn occurs**
Try these updadtes for sklearn in your terminal:

```pip install --upgrade scikit-learn```

```pip install --upgrade imbalanced-learn```

### This function calculates g at a point defined in SWI Cartesians

In [ ]:
def g_at_point(x,y,z):
    # usage for a single point:
    pos = np.asarray([[x,y,z]])
    vx,vy,vz = interpvall.predict(pos).T
    #vx,vy,vz
        
    g_vec = np.asarray([vx[0],vy[0],vz[0]])
    return g_vec

In [ ]:
# Test/example:
# -
# Halfway between the nose of the bowshock and the magnetopause
r_bs = msh.bow_shock(0,0)
r_mp = msh.magnetopause(0,0)
middle_dist = co.mag(np.asarray(r_bs)-np.asarray(r_mp)) # np.asarray makes this into a array [] instead of a tuple ()

halfway_cart_point = np.asarray(r_mp) + np.array([middle_dist/2,0,0])

# g-calculation
g = g_at_point(halfway_cart_point[0],halfway_cart_point[1],halfway_cart_point[2])
print('The g-value at the midpoint at the nose is: ', g)
print('the magnitude is: ', co.mag(g))

## My Modules <a name="MyModules"></a>
Here I will demonstrate the uses of the functions I have written in these modules. **Before using these modules you will need to change the file paths to where your files are stored**

### Coordinate_Systems.py <a name="Coordinatesystems"></a>

This module contains the following functions. For more detail on each function, please open the .py file for notes on each function and there definitions.

Module Contents and there uses:
     
   - Imports

   - Vectors:
       - mag | **Calculates magnitude of a vector (for tuples,lists and arrays)**
   
   - Angle Conversions:
       - deg2rad | **converts degrees to radians**
       - rad2deg | **converts radians to degrees**
     
   - GSE Coordinates:
       - spher2cart_GSE | **Converts GSE spherical coordinates to GSE cartesians: $(r,\theta,\varphi)\rightarrow(x,y,z)$**
       - cart2spher_GSE | **Converts GSE cartesians to GSE sphericals: $(x,y,z)\rightarrow(r,\theta,\varphi)$**
   
   
   - SWI Coordinates:
       - SWI Spherical polar transforms:
           - spher2cart_SWI | **Converts SWI spherical coordinates to SWI cartesians: $(r,\theta,\varphi)\rightarrow(x,y,z)$**
           - cart2spher_SWI | **Converts SWI cartesians to SWI sphericals: $(x,y,z)\rightarrow(r,\theta,\varphi)$**

   - SWI and GSE Transformations:
       - GSE2SWI | **Transforms a GSE defined vector to SWI vector: $(x_{GSE},y_{GSE},z_{GSE})\rightarrow(x_{SWI},y_{SWI},z_{SWI})$, given input:
       ```(GSE vector,swi_base)```, where ```swi_base``` is defined by its function**
       
       - SWI2GSE | **Transforms a SWI defined vector to GSE vector: $(x_{SWI}yx_{SWI}zx_{SWI})\rightarrow(x_{GSE},y_{GSE},z_{GSE})$, given input:
       ```(SWI vector,swi_base)```, where ```swi_base``` is defined by its function**

   - Welle's coordinates Module functions:
       - swi_base | **This function takes in the values of the solarwind bulk velocity and solar wind magnetic field in GSE coordinates. Input: $(u_x,u_y,u_z,B_x,B_y,B_z)$**

---

Examples of uses are below (Specifically the transfomations)

In [ ]:
# Imports required for Coordinate_systems
import numpy as np

# Import module
import Coordinate_systems as co

In [ ]:
# Magnitude of a vector
vec = np.array([1,2,3])
print(co.mag(vec))

### Example for the use of these functions
Here I find the SWI basis and transform a vector defined in the GSE basis and transform it into the SWI basis.

In [ ]:
# Having a vector in GSE and converting to SWI
# Initial GSE vector (example uses first Berdichevsky solar wind vector)

u_sw_GSE = np.array([-443,-76,25]) # Bulk

# Magnetic field (for first shock in Berdichevsky) in spherical GSE:
# 'phi_B': (103,5), 'theta_B': (15,11), 'mag_B': (10,1.5)

# Convert degrees to radians
theta_r = co.deg2rad(15)
phi_r   = co.deg2rad(103)
mag_B = 10

# Convert B-field sphericals to cartesians 
B_sw_GSE = co.spher2cart_GSE(mag_B,theta_r,phi_r) # make sure in radians!!

# ---------- Transforming coordinate systems
Basis_matrix = co.swi_base(u_sw_GSE[0],u_sw_GSE[1],u_sw_GSE[2],B_sw_GSE[0],B_sw_GSE[1],B_sw_GSE[2])

# Use this matrix to transform a vector defined in GSE

vector_in_GSE = np.array([1,2,3]) # some vector in GSE

# Transfom to SWI
vector_in_SWI = co.GSE2SWI(vector_in_GSE,Basis_matrix)

print('vector in GSE: ', vector_in_GSE)
print('vector in SWI: ', vector_in_SWI)
print('showing the the magnitude is conserved (as it should be for coordinate transfroms of vectors)')
print('GSE mag', co.mag(vector_in_GSE))
print('SWI mag', co.mag(vector_in_SWI))

### Shocks_propagation.py <a name="shocks_prop_mod"></a>

This module contains some useful functions when analysing shocks. Again you will need to change the paths to files in the modules.

For more detail on the functions please look at the module file as it contains descriptions and instructions on what the functions do.

Module Contents:

   - Imports

   - Magnetosphere Module

   - Hypotheses
       - vec_add_hyp | **Will calculate the shock velocity vector given a point in the MSH**
   
   - Random Point inside Magnetosheath
       - rand_msh_point_SWI | **This provides a fairly sampled point inside the MSH for a given $\theta$ limit**

   - Propegation functions | **These are old functions, however can come in handy when wanting to know the points along the path taken in a straght line inside the MSH**
       - propegation_V1
       - propegation_V2

## Important!
None of these functions are accounting for **dynamic pressure** as when I originally wrote them I was not accounting for dynamic pressure!

This can be easily fixed by adding in a input for the density and solar wind speed (in SI units) and then coounting for the DP in the ```msh.``` functions!

**Imports**

In [ ]:
# Imports
import Coordinate_systems as co
#
#
import numpy as np
#
# -----------------------------------------------------
# These Imports are used specifically for Welle's Code:
#
# - - - - - - - - - For g-calculation - - - - - - - - -
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
#
# - - - - For r_magnetosheath and r_magnetopause - - - 
#
# This will need changing for user of the module
# ------------ PASTE THESE IN MAIN CODE ---------------
import sys
sys.path.append('/Users/christopherbutcher/Downloads/Summer Masterclass/space')
import space
from space.models import planetary as smp
from space import smath as sm
from space.coordinates import coordinates as scc
from space import utils as su

In [ ]:
import Shocks_propagation as shks

**Example - Random point inside MSH**

In [ ]:
# Random point given some theta_max
theta_max = 100
rand_point = shks.rand_msh_point_SWI(theta_max) # theta_max in degrees
print('Here is a random point in the magnetosheath given some maximum theta: ', rand_point)

# You might want to know what the minimum x-value sampled is...
X_min = -co.mag(msh.bow_shock(co.deg2rad(theta_max),0))*np.cos(np.pi-co.deg2rad(theta_max))

print('The minimum x-value is therfore: ',X_min)

## V2_Rand_prop_points.py <a name='rand_point'></a>

This is a python file that you can run and find the f-value distribution for randomly sampled points (for non-inclined shocks) inside the magnetosheath. You will need to enter 3 things:
1. The number of points to be sampled
2. The path intigral step size
3. The maximum theta value for which the points can be sampled in (it will then say the minimum x-value samples)

To run this file you will require all the modules and files previously mentioned (the pkl file ect). You will also need to change the paths to these files as required. To run in the terminal:
```python3 V2_Rand_prop_points.py```

## Berdichevsky Shock json file <a name='shocks_file'></a>
Here I have the code that makes the Berdichevsky data into a json file.

I then have some code that will convert these values of data into GSE cartesians or alternitavly into SWI cartesians (see below)


In [ ]:
# ===========================================================================================================================
# =================================== This can be removed after script has been ran once ====================================
# ===========================================================================================================================

import json

Shocks_from_Berdichevsky = {
        '1': {'Vs_up':(120,20), 'Vs': (-443,-76,25), 'phi_shock': (214,3), 'theta_shock': (17,3), 'phi_B': (103,5), 'theta_B': (15,11), 'mag_B': (10,1.5)},
        '2': {'Vs_up':(61,10), 'Vs': (-376,-10,9), 'phi_shock': (190,12), 'theta_shock': (10,8), 'phi_B': ('Na','Na'), 'theta_B': ('Na','Na'), 'mag_B': ('Na','Na')},
        '3': {'Vs_up':(94,7), 'Vs': (-474,-8,33), 'phi_shock': (202,2), 'theta_shock': (44,2), 'phi_B': (270,1), 'theta_B': (17,0.5), 'mag_B': (4.1,0.2)},
        '5': {'Vs_up':(90,5), 'Vs': (-375,4,-30), 'phi_shock': (170,6), 'theta_shock': (-15,4), 'phi_B': (84,2), 'theta_B': (-41,4), 'mag_B': (3.9,0.2)},
        '10': {'Vs_up':(80,8), 'Vs': (-386,-50,15), 'phi_shock': (206,14), 'theta_shock': (2,2), 'phi_B': (176,19), 'theta_B': (19,14), 'mag_B': (2.0,0.4)},
        '11': {'Vs_up':(75,10), 'Vs': (-381,12,44), 'phi_shock': (147,7), 'theta_shock': (30,2), 'phi_B': (290,12), 'theta_B': (20,14), 'mag_B': (2.1,0.5)},
        '12': {'Vs_up':(73,8), 'Vs': (-385,-6,-20), 'phi_shock': (185,5), 'theta_shock': (-21,5), 'phi_B': (250,4), 'theta_B': (20,13), 'mag_B': (6.6,1.0)},
        '13': {'Vs_up':(110,20), 'Vs': (-408,-36,-18), 'phi_shock': (202,9), 'theta_shock': (-9,7), 'phi_B': (85,68), 'theta_B': (0,90), 'mag_B': (0.7,1.2)},
        '14': {'Vs_up':(137,5), 'Vs': (-467,72,52), 'phi_shock': (128,0), 'theta_shock': (21,0), 'phi_B': ('Na','Na'), 'theta_B': ('Na','Na'), 'mag_B': ('Na','Na')},
        '15': {'Vs_up':(110,25), 'Vs': (-433,-30,84), 'phi_shock': (120,7), 'theta_shock': (48,3), 'phi_B': (104,3), 'theta_B': (0,6), 'mag_B': (4.4,0.1)},
        '16': {'Vs_up':(60,10), 'Vs': (-342,1,-84), 'phi_shock': (182,10), 'theta_shock': (-48,12), 'phi_B': (87,3), 'theta_B': (-10,3), 'mag_B': (3.6,0.2)},
        '17': {'Vs_up':(39,7), 'Vs': (-346,-24,-1), 'phi_shock': (162,5), 'theta_shock': (13,3), 'phi_B': (316,4), 'theta_B': (13,8), 'mag_B': (3.4,0.4)},
        '18': {'Vs_up':(100,10), 'Vs': (-416,-85,-54), 'phi_shock': (205,4), 'theta_shock': (-15,2), 'phi_B': (141,16), 'theta_B': (-9,14), 'mag_B': (6.5,1.6)},
# Prev in data (Reverse Shock)'19': {'Vs_up':(125,15), 'Vs': (-509,-28,71), 'phi_shock': (,), 'theta_shock': (,)},
        '20': {'Vs_up':(52,8), 'Vs': (-391,-11,7), 'phi_shock': (188,6), 'theta_shock': (16,12), 'phi_B': (17,0.6), 'theta_B': (23,2), 'mag_B': (4.1,0.2)},
        '21': {'Vs_up':(45,12), 'Vs': (-422,-3,20), 'phi_shock': (185,7), 'theta_shock': (45,15), 'phi_B': (352,6), 'theta_B': (11,3), 'mag_B': (3.0,0.2)},
        '22': {'Vs_up':(50,15), 'Vs': (-359,-9,45), 'phi_shock': (156,24), 'theta_shock': (55,7), 'phi_B': (4.7,0.1), 'theta_B': (10,3), 'mag_B': (2.7,0.3)},
        '23': {'Vs_up':(52,10), 'Vs': (-380,-20,22), 'phi_shock': (202,2), 'theta_shock': (25,8), 'phi_B': (112,2), 'theta_B': (25,2), 'mag_B': (4.4,0.2)},
        '24': {'Vs_up':(70,7), 'Vs': (-318,-49,49), 'phi_shock': (213,3), 'theta_shock': (50,3), 'phi_B': (138,5), 'theta_B': (15,2), 'mag_B': (5.7,0.4)},
        '25': {'Vs_up':(75,15), 'Vs': (-478,-5,-6), 'phi_shock': (168,5), 'theta_shock': (4,4), 'phi_B': (253,3), 'theta_B': (-15,7), 'mag_B': (7.6,0.8)},
        '26': {'Vs_up':(60,20), 'Vs': (-355,-32,-22), 'phi_shock': (187,10), 'theta_shock': (-24,5), 'phi_B': (143,10), 'theta_B': (9,5), 'mag_B': (2.0,0.2)},
        '28': {'Vs_up':(55,16), 'Vs': (-329,-11,15), 'phi_shock': (169,3), 'theta_shock': (21,2), 'phi_B': (149,3), 'theta_B': (36,10), 'mag_B': (4.7,0.6)},
        '29': {'Vs_up':(40,10), 'Vs': (-370,-20,0.5), 'phi_shock': (194,10), 'theta_shock': (-6,7), 'phi_B': (164,19), 'theta_B': (26,27), 'mag_B': (2.2,0.8)},
        '30': {'Vs_up':(75,8), 'Vs': (-439,-31,-18), 'phi_shock': (200,3), 'theta_shock': (-30,3), 'phi_B': (161,4), 'theta_B': (8,7), 'mag_B': (2.2,0.3)},
        '32': {'Vs_up':(115,13), 'Vs': (-632,44,-26), 'phi_shock': (197,26), 'theta_shock': (5,5), 'phi_B': (204,9), 'theta_B': (30,11), 'mag_B': (3.6,0.6)},
        '33': {'Vs_up':(77,6), 'Vs': (-393,-17,24), 'phi_shock': (184,1), 'theta_shock': (-10,1), 'phi_B': (348,6), 'theta_B': (29,8), 'mag_B': (4.5,0.5)},
        '34': {'Vs_up':(53,3), 'Vs': (-337,-16,43), 'phi_shock': (194,1), 'theta_shock': (52,0.5), 'phi_B': (72,2), 'theta_B': (29,8), 'mag_B': (3.1,0.4)},
        '35': {'Vs_up':(72,10), 'Vs': (-378,-14,53), 'phi_shock': (199,8), 'theta_shock': (58,3), 'phi_B': (291,17), 'theta_B': (-20,24), 'mag_B': (2.0,0.6)},
        '36': {'Vs_up':(91,10), 'Vs': (-359,-5.5,77), 'phi_shock': (160,7), 'theta_shock': (51,5), 'phi_B': (330,20), 'theta_B': (45,12), 'mag_B': (7.0,1.5)},
        '38': {'Vs_up':(64,6), 'Vs': (-372,-17,43), 'phi_shock': (217,2), 'theta_shock': (49,2), 'phi_B': (14,0.5), 'theta_B': (-11,32), 'mag_B': (2.0,0.8)},
        '39': {'Vs_up':(121,10), 'Vs': (-422,-70,-58), 'phi_shock': (208,0.5), 'theta_shock': (-19,1), 'phi_B': (113,2), 'theta_B': (-17,5), 'mag_B': (7.2,0.5)},
        '40': {'Vs_up':(58,6), 'Vs': (-344,16,-32), 'phi_shock': (190,20), 'theta_shock': (-15,12), 'phi_B': (142,9), 'theta_B': (-28,11), 'mag_B': (3.4,0.8)},
        '41': {'Vs_up':(73,7), 'Vs': (-363,-6,-32), 'phi_shock': (185,2), 'theta_shock': (-20,1), 'phi_B': (103,18), 'theta_B': (37,16), 'mag_B': (4.8,1.3)},
        '42': {'Vs_up':(60,9), 'Vs': (-339,-10,-36), 'phi_shock': (176,14), 'theta_shock': (-26,7), 'phi_B': (158,1), 'theta_B': (-8,21), 'mag_B': (2.5,0.5)}
                            }

with open('shocks_from_Berdichevsky.json', 'w') as f:
    json.dump(Shocks_from_Berdichevsky, f)

### Converting data into useful arrays

Below I have some code that will change this data into a more useful array format where all arrays are in cartesians (GSE or SWI).

1. GSE cartesian coordinate arrays
2. SWI cartesian coordinate arrays



### 1. GSE cartesian coordinate arrays

In [ ]:
# ============================================================================================================================
# ========================== Storing values of solar wind and shock data in GSE coordinates (from JSON file) =================
# ============================================================================================================================
#
# Storing values of solar wind and shock data in GSE coordinates (from JSON file)
#
v_sw_cart_GSE_shocks   = []
u_sw_cart_GSE_shocks   = []
B_field_GSE_shocks     = []
shock_index            = [] # This allows you to give later refernece to which shock in Berdichevsky produced this


for shock in Shocks_from_Berdichevsky:
    
    
    # -------------- B-field Direction ------------
    #
    # Some do not have a calculated B-field
    if Shocks_from_Berdichevsky[shock]['mag_B'][0]=='Na':
        
        pass
        
    else:
        
        # ---------------------------------- Solar wind --------------------------
        # solar wind GSE cartisian vector
        u_sw_GSE = Shocks_from_Berdichevsky[shock]['Vs']
        
        # Storing solar wind data
        u_sw_cart_GSE_shocks.append(np.asarray(u_sw_GSE))
        
        
        # ---------------------------------- Shock data --------------------------
        # shock normal vector (in GSE spherical polars)     + solar wind
        v_sw_mag = Shocks_from_Berdichevsky[shock]['Vs_up'][0] + co.mag(u_sw_GSE)
        
        # Shock normal in GSE cartisans (conversion)
        v_sw_cart_GSE = co.spher2cart_GSE(v_sw_mag,co.deg2rad(Shocks_from_Berdichevsky[shock]['theta_shock'][0]),co.deg2rad(Shocks_from_Berdichevsky[shock]['phi_shock'][0]))
        
        # Storing v_sw directions
        v_sw_cart_GSE_shocks.append(v_sw_cart_GSE)
        
        # ----------------------------------- Magnetic field data ----------------
        
        B_field_GSE_shocks.append(co.spher2cart_GSE(Shocks_from_Berdichevsky[shock]['mag_B'][0],co.deg2rad(Shocks_from_Berdichevsky[shock]['theta_B'][0]),co.deg2rad(Shocks_from_Berdichevsky[shock]['phi_B'][0])))
        
        
        # ------------------ Shock Index -----------------------------------------
        # Each shocks index
        shock_index.append(int(shock))

### 2. SWI cartesian coordinate arrays

In [ ]:
# ==========================================================================================================================
# ===================================== Transforming GSE coordinates to SWI coordinates ====================================
# ==========================================================================================================================

u_sw_cart_SWI_shocks      = []
v_sw_cart_SWI_shocks      = []
#initial_cart_shock_points = []

for i in range(0,len(shock_index),1):
    
    #                                     x_GSE                   y_GSE                      z_GSE                          B_x                     B_y                          B_z
    SWI_base_matrix = co.swi_base(u_sw_cart_GSE_shocks[i][0],u_sw_cart_GSE_shocks[i][1],u_sw_cart_GSE_shocks[i][2],B_field_GSE_shocks[i][0],B_field_GSE_shocks[i][1],B_field_GSE_shocks[i][2])
    
    
    # ------------------------- Solar wind vectors to SWI coordinates ------------------
    u_sw_cart_SWI = co.GSE2SWI(u_sw_cart_GSE_shocks[i],SWI_base_matrix)
    
    u_sw_cart_SWI_shocks.append(u_sw_cart_SWI)
    
    # ------------------------------- Shock normal vectors -----------------------------
    v_sw_cart_SWI = co.GSE2SWI(v_sw_cart_GSE_shocks[i],SWI_base_matrix)
    
    v_sw_cart_SWI_shocks.append(v_sw_cart_SWI)

These arrays now contain the values in GSE or SWI cartesian coordinate arrays. This is useful as you often wish to use SWI coordinates (due to the Bow shock).